In [1]:
# roll 5 dice
# you can roll each die 3 times

# select among 13 categories
# upper section gives the number of points corresponding to a die set of one kind
# if you get more than 63 in upper section, you get bonus of 
# the lower section contains
# Small straight - 4 in a sequence 30p
# Large straight - 5 in a sequence 40p
# full house - 3 of one and 2 of another 25p
# yatzhe five of a kind - 50p
# second yatzee scores a 100 p, but it must be put in the appropriate box
# chance - put anything, you get total points of die faces
# you have the option to put roll in a cateogry

http://www.yahtzee.org.uk/rules.html

SyntaxError: invalid syntax (<ipython-input-1-0e144902f88a>, line 16)

In [123]:
import random

# YATZEE GAME
# An implementation of the game Yatzee
# Goal is to create simulate a game of yatzee
# where the player can can use different strategies
# to play the game. Later we want to simulate the outcome of different strategies.
#
# Current version is a simplification to get overall structure working
# 6 dice are rolled
# The user always discards the first 3, which are replaced with 3 new ones
# This is repeated 2 times



# A die that returns a number between 1 and 6
class Dice:
    
    def __init__(self):
        pass
    
    def roll(self):
        return random.choice([1,2,3,4,5,6])
    

# the main class of the actual game
class Yatzee:
    
    def __init__(self, player):
        self.player = player
        # to keep track of the score for the player
        self.player_score = 0
        
        self.die_set = []
        
        # the yatzee score card
        self.player_scores = {
            
            'ONES' : None, 
            'TWOS' : None, 
            'THREES' : None, 
            'FOURS' : None, 
            'FIVES' : None, 
            'SIXES' : None,
            'STRAIGHT' : None, 
            'FULL_STRAIGHT' : None, 
            'HOUSE' : None, 
            'YATZEE' : None
        }
        
            # returns the scoring options that are still available in the game
    def get_available_scores(self):
        
        return [k for k, v in self.player_scores.items() if v is None]
    
    # start a game
    def play(self):
        
        
        print("****Game started****")
        
        while len(self.get_available_scores()) > 0:
            
            print('--------NEW ROUND---------')
        
            self.die_set = self.roll_die(6)

            print("First dice roll")
            print(self.die_set)

            for i in range(2):
                player_decision = player.select_dice(self.die_set)
                # returns a new set of 6 die that the player can choose from
                self.update_die_set(player_decision)


            # at this stage, we should now have 6 dice with a score

            # after three rounds, we go on to select our scoring mechanism
            # the decision is based on the last set of die, and the available scores
            # this should return the selected score, as an index in the array of available options

            available_scores = self.get_available_scores()
            
            
            print("AVAILABLE SCORES")
            print(available_scores)

            player_selected_score = player.choose_score(self.die_set, available_scores)

            print("Player selected")
            print(self.get_available_scores()[player_selected_score])

            print('Updating scores...')

            self.update_yatzee_score(player_selected_score)

            print("Current scoring")
            print(self.get_player_score())
        
        print("******************************************")
        print("No scoring options available. Game ending")
        print("Total score ", self.get_player_score())
            
        
        
        
        # this calculates the score given a set of die and a selected score
    def get_player_score(self):
        
        return sum([v for k,v in self.player_scores.items() if v is not None])
    
    # this should calulcate the score given the die set
    # and a player selection
    def update_yatzee_score(self, player_selection):
        
        
        ## add check to see if the option is valid or not
        
        print("Selecting scoring option")
        
        chosen_option = self.get_available_scores()[player_selection]
        
        if chosen_option in ['ONES', 'TWOS', 'THREES','FOURS', 'FIVES', 'SIXES']:
            # should only calculate for the dice of the correct count/type
            self.player_scores[chosen_option] = sum(self.die_set)
        elif chosen_option == 'STRAIGHT':
            self.player_scores[chosen_option] = 30
        elif chosen_option == 'FULL_STRAIGHT':
            self.player_scores[chosen_option] = 40
        elif chosen_option == 'HOUSE':
            self.player_scores[chosen_option] = 25
        elif chosen_option == 'YATZEE':
            self.player_scores[chosen_option] = 50
        
        
        
    # each time the user returns his new option, we replace those die with 3 new
    # and make sure that the user never 
    
    # take the users die decision and roll the new the new dice
    def update_die_set(self, player_decision):
        discarded_dice = []
        # remove the user selection from the set of die
        for dice_to_throw_again in player_decision:
            discarded_dice.append(self.die_set.pop(dice_to_throw_again))
            
        print("User discards:")
        print(discarded_dice)
        
        # calculate the number of new die to throw
        num_die = 6 - len(self.die_set)
        
        # throw new die
        new_die = self.roll_die(num_die)
        
        # add them to the set of die
        self.die_set.extend(new_die)
        
        print("Dice after re-throw:")
        print(self.die_set)
        
    def roll_die(self, num_die):
        
        dice_result = []
        
        for d in range(num_die):
            dice = Dice()
            dice_result.append(dice.roll())
            
            
        return dice_result
    

        
    
class Player:
    
    def __init__(self):
        pass
    
    # method for selecting an option
    # it should return a structure that the game
    # can use to understand which dices to roll again
    # such as [0,1,0,1,0,0], where 1 = keep, 0 = throw
    
    # should probably be related to the scores
    
    def select_dice(self,dice_set):
        
        die_to_throw = []
        
        # just throw away the three first ones for now
        die_to_throw = [0,1,2]

        return die_to_throw
    
    # the users algorithm to select the score
    # looks at all the available alternatives and chooses one of them
    # returns a score selection
    def choose_score(self, die, available_scores):
        # always select the first available one
        return 0
        
player = Player()

yatzee = Yatzee(player)

In [124]:
yatzee.play()

****Game started****
--------NEW ROUND---------
First dice roll
[1, 2, 5, 1, 6, 6]
User discards:
[1, 5, 6]
Dice after re-throw:
[2, 1, 6, 2, 6, 3]
User discards:
[2, 6, 6]
Dice after re-throw:
[1, 2, 3, 5, 1, 1]
AVAILABLE SCORES
['ONES', 'TWOS', 'THREES', 'FOURS', 'FIVES', 'SIXES', 'STRAIGHT', 'FULL_STRAIGHT', 'HOUSE', 'YATZEE']
Player selected
ONES
Updating scores...
Selecting scoring option
Current scoring
13
--------NEW ROUND---------
First dice roll
[3, 2, 4, 5, 4, 2]
User discards:
[3, 4, 4]
Dice after re-throw:
[2, 5, 2, 5, 3, 3]
User discards:
[2, 2, 3]
Dice after re-throw:
[5, 5, 3, 6, 3, 5]
AVAILABLE SCORES
['TWOS', 'THREES', 'FOURS', 'FIVES', 'SIXES', 'STRAIGHT', 'FULL_STRAIGHT', 'HOUSE', 'YATZEE']
Player selected
TWOS
Updating scores...
Selecting scoring option
Current scoring
40
--------NEW ROUND---------
First dice roll
[2, 5, 5, 2, 3, 3]
User discards:
[2, 5, 3]
Dice after re-throw:
[5, 2, 3, 4, 4, 3]
User discards:
[5, 3, 4]
Dice after re-throw:
[2, 4, 3, 3, 6, 5]
AVAIL

In [19]:
arr = [1,2,3,5,1]

arr_2 = [1,2,3]

arr.extend(arr_2)

sum(arr)


18

In [97]:
player_scores = {
    'ONES' : 2, 
    'TWOS' : None, 
    'THREES' : 3, 
    'FOURS' : None, 
    'FIVES' : None, 
    'SIXES' : None,
    'BONUS' : None,
    'STRAIGHT' : None,
    'FULL STRAIGHT' : None, 
    'HOUSE' : None, 
    'YATZEE' : None
}

5

In [106]:
player_scores

TypeError: unsupported operand type(s) for +: 'int' and 'NoneType'

In [55]:
player_scores['ONES'] = 4

In [56]:
player_scores['ONES']

4

In [57]:
player_scores

{'ONES': 4,
 'TWOS': None,
 'THREES': None,
 'FOURS': None,
 'FIVES': None,
 'SIXES': None,
 'BONUS': None,
 'STRAIGHT': None,
 'FULL STRAIGHT': None,
 'HOUSE': None,
 'YATZEE': None}

In [ ]:
None in player_scores.values()

In [ ]:
{k for k, v in player_scores.items() if v is None}

In [ ]:
scores = [k for k, v in player_scores.items()]

In [ ]:
if scores[0] in ['TWOS', 'THREES','FOURS', 'FIVES', 'SIXES', 'BONUS','STRAIGHT', 'FULL STRAIGHT', 'HOUSE',  'YATZEE']:
    print("Hello")
else:
    print("Not hello")

In [60]:
'ONES' in ['ONES', 'TWOS', 'THREES','FOURS', 'FIVES', 'SIXES']

True